In [16]:
import os
import subprocess, shlex
import simplejson as json
import pandas as pd
from dashboard.data_load import load_mouse_blastp_results, load_esmfold
from pathlib import Path

In [4]:
from dashboard.etl import CACHE_DIR, PROTEIN_TOOLS_PATH


In [11]:
cache_dir = Path('/home/ubuntu/repos/dashboard/cache/')

In [8]:
output_prefix = CACHE_DIR.joinpath('protein_data')

In [13]:
subprocess.run(shlex.split(f'docker run  --rm -v {output_prefix}:/data -v /efs/databases/blast:/db ncbi/blast blastp -task blastp-fast -outfmt 15 -db /db/mouse.protein.faa -query /data/protein_tools_input.fasta -max_target_seqs 20 -out /data/blastp.results.json'))


CompletedProcess(args=['docker', 'run', '--rm', '-v', '/home/ubuntu/repos/dashboard/cache/protein_data:/data', '-v', '/efs/databases/blast:/db', 'ncbi/blast', 'blastp', '-task', 'blastp-fast', '-outfmt', '15', '-db', '/db/mouse.protein.faa', '-query', '/data/protein_tools_input.fasta', '-max_target_seqs', '20', '-out', '/data/blastp.results.json'], returncode=0)

In [22]:
cmd = f'docker run --gpus all -it -v /home/ubuntu/repos/protein_tools:/opt/openfold -v {output_prefix}:/data 328315166908.dkr.ecr.us-west-2.amazonaws.com/esmfold:latest python /opt/openfold/run_batch_fasta.py /data/protein_tools_input.fasta /data/esmfold.jsonlines'
subprocess.run(shlex.split(cmd))

the input device is not a TTY


CompletedProcess(args=['docker', 'run', '--gpus', 'all', '-it', '-v', '/home/ubuntu/repos/protein_tools:/opt/openfold', '-v', '/home/ubuntu/repos/dashboard/cache/protein_data:/data', '328315166908.dkr.ecr.us-west-2.amazonaws.com/esmfold:latest', 'python', '/opt/openfold/run_batch_fasta.py', '/data/protein_tools_input.fasta', '/data/esmfold.jsonlines'], returncode=1)

In [23]:
print(cmd)

docker run --gpus all -it -v /home/ubuntu/repos/protein_tools:/opt/openfold -v /home/ubuntu/repos/dashboard/cache/protein_data:/data 328315166908.dkr.ecr.us-west-2.amazonaws.com/esmfold:latest python /opt/openfold/run_batch_fasta.py /data/protein_tools_input.fasta /data/esmfold.jsonlines


In [17]:
esmfold = load_esmfold()


2023-12-04 20:56:16.680 
  command:

    streamlit run /opt/conda/envs/veliadash_updated/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-12-04 20:56:16.681 No runtime found, using MemoryCacheStorageManager


In [19]:
len(esmfold.keys())

13

In [20]:
esmfold.keys()

dict_keys(['MVQGSCWAAGKRGCTS', 'MISAHCDLCFLGSRILLPQPPK', 'MRSRVHTPRGGEALNRETA', 'MPRINREGRRWLFSCLIVFLLSELESHEGEQQKTTN', 'MRFFRLTFKCFVDCF', 'MILQSLLFLQRLLMISTKPAVVLLWPLLKKVENTLMQHVHPNLPA', 'MQHPGEPTCILLARCLECNTTCESENLPRPPAMD', 'MTHNSGSGPGTSNIIK', 'MLEGGFRRKMILCILSLHPNF', 'MANDRRRAQTGPCYDLSWSGSD', 'MQEVSRKGRTPGFEQNFGL', 'MHYLIKRRMDLLFLQLVFQELI', 'MAEIRTLHFAACALRPQNLLYPT'])

In [14]:
with open(cache_dir.joinpath('protein_data', 'blastp.results.json'), 'r') as fopen:
    blastp = json.load(fopen)
    blastp = blastp['BlastOutput2']